In [ ]:
import cmocean as cmo
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import signal
from eofs.standard import Eof
from eofs.examples import example_data_path
from xmitgcm import open_mdsdataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read data and bathy
outdir = '/data/SO2/sio-kramosmusalem/exp06_512x612x100_ORL_SVB/01_SVB_febTS_output/'
outdir2 = '/data/SO2/sio-kramosmusalem/exp06_512x612x100_ORL/01_noSVB_febTS/'

levels = [1,   2,  3,  4,  5,  6,  7,  8,  9, 10, 
          11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25,
          26, 27, 28, 29, 30, 31,
          32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
          45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,              
          58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 
          74, 79, 84, 89, 94, 99,]
ds = open_mdsdataset(outdir, prefix=['eta','dynVars'], levels=levels)
ds2 = open_mdsdataset(outdir2, prefix=['eta','dynVars'], levels=levels)

In [ ]:
depth = ds2.Depth[:]
lats = ds2.YC[:,0].data
lons = ds2.XC[0,:].data-360
hFacC = ds2.hFacC.data
hfac = np.ma.masked_values(hFacC, 0)
mask = np.ma.getmask(hfac)

ssh = ds.ETAN[:].data-ds2.ETAN[:].data
mask_ssh = np.expand_dims(mask[0,:,:], axis=0) + np.zeros_like(ssh)
ssh_masked = np.ma.masked_array(ssh, mask=mask_ssh)

In [ ]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
coslat = np.cos(np.deg2rad(lats))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(ssh_masked, weights=wgts) # I tried with weights=None and I got the same result. KRM

In [ ]:
# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
eof = solver.eofs(neofs=3,eofscaling=1)
eofCorr = solver.eofsAsCorrelation(neofs=3)
pc = solver.pcs(npcs=3, pcscaling=1)
varFact = solver.varianceFraction(neigs=3)

In [ ]:
# Plot the leading PC time series.
sns.set_context('talk')
times = np.arange(0,720)*600
fig,ax = plt.subplots(1,1, figsize=(14,4))
ax.axhline(0, color='0.8')
ax.axvline(3.6, color='0.8', linestyle='--')
ax.plot(times/3600, pc[:,0], color='purple', linewidth=2, label='PC1')
ax.plot(times/3600, pc[:,1], color='orange', linewidth=2, label='PC2')
ax.plot(times/3600, pc[:,2], color='green', linewidth=2, label='PC3')
ax.set_title('PC Time Series')
ax.set_xlabel('Hours')
ax.set_ylabel('Normalized Units')
ax.set_xlim(0,120)
#plt.ylim(-3, 3)
plt.legend()
plt.show()

In [ ]:
# Plot the leading EOF expressed as correlation in the Pacific domain.
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,7), sharey=True)
ax1.set_ylabel('Lat')
for nmode, ax in zip([0,1,2], [ax1,ax2,ax3]):
    ax.set_facecolor('tan')
    cn = ax.pcolormesh(lons, lats, np.ma.masked_array(eof[nmode].squeeze(),mask=mask[0,:,:]), 
                       cmap=cmo.cm.curl, vmin=-0.6, vmax=0.6)
    cb = plt.colorbar(cn, ax=ax, orientation='horizontal', pad=0.15)
    ax.contour(lons, lats, eof[nmode].squeeze(), levels=[0],colors=['0.2'])
    ax.contour(lons,lats, depth, levels=[0], colors='k')
    cb.set_label('unscaled EOF')
    ax.set_title('EOF%d, variance explained=%1.3f' %(nmode+1,varFact[nmode]), fontsize=14)
    ax.set_aspect(1)
    ax.set_xlabel('Lon')